In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

brewery_2022_interpolated = pd.read_csv("Data/brewery_production_2022_interpolated.csv")
brewery_2017_2019 = pd.read_csv("Data/brewery_production_2017_2019.csv")
wa_breweries_with_ratings = pd.read_csv("Data/wa_breweries_with_ratings.csv")
wa_breweries_with_ratings = wa_breweries_with_ratings[["brewery", "rating", "total_visits", "unique_visits", "leg_district"]]
wa_breweries_with_ratings['customer_loyalty'] = wa_breweries_with_ratings['total_visits'] / wa_breweries_with_ratings['unique_visits']

In [ ]:
brewery_2022_interpolated.columns

In [ ]:
brewery_2022_interpolated["closed_since_2022"].value_counts()

In [ ]:
brewery_2022_total = pd.DataFrame({
    "brewery_name": brewery_2022_interpolated["brewery"], 
    "annual_production": brewery_2022_interpolated["total_annual"],
    "year": 2022,
    "estimate": 1
})


In [ ]:
brewery_2017_2022 = pd.concat([brewery_2022_total, brewery_2017_2019])

In [ ]:
def num_handler(s):
    if s == "DNP" or s == "Do Not Publish":
        return np.nan
    if type(s) == str:
        s = s.replace(",", "")
    return float(s)

In [ ]:
brewery_2017_2022

In [ ]:
brewery_production_trend = pd.DataFrame(brewery_2017_2022.groupby("brewery_name")["annual_production"].agg(lambda x: np.nan if len(x) == 1 else np.polyfit(np.arange(len(x)), np.array(list(map(lambda v: num_handler(v) ,x.values[::-1]))), 1)[0] / num_handler(x.iloc[-1])))
brewery_production_trend.rename(columns={"annual_production": "production_trend"}, inplace=True)
brewery_production_trend.fillna(0, inplace=True)
brewery_production_size = pd.DataFrame(brewery_2017_2022.groupby("brewery_name")["annual_production"].agg(lambda x: np.sum(list(map(lambda v: num_handler(v) ,x.values)))/len(x)))
# categorize brewery production by percentile
brewery_production_size.fillna(0, inplace=True)
brewery_production_size.rename(columns={"annual_production": "production_size"}, inplace=True)
brewery_production_stat = pd.concat([brewery_production_trend, brewery_production_size], axis=1)

In [ ]:
rawDF = pd.merge(brewery_2022_interpolated, brewery_production_stat, left_on="brewery", right_on="brewery_name", how="inner")
rawDF = pd.merge(rawDF, wa_breweries_with_ratings, left_on="brewery", right_on="brewery", how="inner")

In [ ]:
rawDF[rawDF['closed_since_2022'] == 1]

In [ ]:
dataDF = rawDF[["brewery", 'brewery_type', 'city', 'county', 'latitude', 'longitude', 'leg_district',
       'year_established', 'guild_member',
       'closed_since_2022', "total_annual", "production_trend", "production_size", "rating", "customer_loyalty", "total_visits"]]
dataDF['total_annual'] = dataDF['total_annual'].fillna(0)
dataDF['year_established'] = dataDF['year_established'].apply(lambda x: 2022 - x + 1)
dataDF['average_visits'] = dataDF['total_visits'] / dataDF['year_established']
dataDF['leg_district'] = dataDF["leg_district"].astype("category")

In [ ]:
dataDF.columns

In [ ]:
dataDF = pd.get_dummies(dataDF[['latitude', 'longitude', 'brewery_type',
       'year_established', 'guild_member',
       'closed_since_2022', "production_trend", "production_size", "rating", "customer_loyalty", "average_visits"]])
dataDF.dropna(inplace=True)

In [ ]:
dataDF.shape

In [ ]:
# find the neighbors' production sizes
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(dataDF[["longitude", "latitude"]])
distances, indices = neigh.kneighbors(dataDF[["longitude", "latitude"]])

# get the production size of neighbors
neighbor_production_size_mean = []
neighbor_production_size_max = []
neighbor_production_size_min = []
neighbor_production_size_std = []
for i in range(len(indices)):
    neighbor_production_size_mean.append(dataDF.iloc[indices[i][1:]]["production_size"].mean())
    neighbor_production_size_max.append(dataDF.iloc[indices[i][1:]]["production_size"].max())
    neighbor_production_size_min.append(dataDF.iloc[indices[i][1:]]["production_size"].min())
    neighbor_production_size_std.append(dataDF.iloc[indices[i][1:]]["production_size"].std())

dataDF["neighbor_production_size_mean"] = neighbor_production_size_mean
dataDF["neighbor_production_size_max"] = neighbor_production_size_max
dataDF["neighbor_production_size_min"] = neighbor_production_size_min
dataDF["neighbor_production_size_std"] = neighbor_production_size_std

In [ ]:
# stats for breweries closed since 2022

# output the aggregated result as figure with seaborn
comparison_stat = dataDF.groupby("closed_since_2022").mean()
comparison_stat_normalized = comparison_stat / comparison_stat.sum()

# plot the comparison
fig = plt.figure(figsize=(15, 10))
sns.set_theme(style="whitegrid")
# horizontal bar plot with closed as red and open as blue and plot each x-axis independently
ax = comparison_stat_normalized.T.plot(kind="barh", color=["blue", "red"])
ax.get_figure().savefig("Viz/Draft2/closed_vs_open.png", bbox_inches='tight')

In [ ]:
dataDF["closed_since_2022"].value_counts()

In [ ]:
dataDF.shape

In [ ]:
X = pd.get_dummies(dataDF.drop(columns=["closed_since_2022", "longitude", "latitude"])).values
y = dataDF["closed_since_2022"].values

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.model_selection as skm
from sklearn.metrics import roc_auc_score

seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# from sklearn.decomposition import PCA

# pca = PCA(n_components=int(np.sqrt(X.shape[1])))
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score
from ISLP import confusion_table

# kfold grid search with skm
# fit a support vector classifier on the training set
svm = SVC()

kfold = skm.KFold(3, 
                  random_state=seed,
                  shuffle=True)
grid = skm.GridSearchCV(svm,
                        {'C':[0.01,0.1,1,5,10, 100], "gamma": [0.01,0.1,1,5,10,100], "kernel":['linear', 'rbf'], "class_weight":['balanced', None]},
                        refit=True,
                        cv=kfold,
                        scoring='roc_auc')
grid.fit(X_train, y_train)
print("grid best params:", grid.best_params_)

best_ = grid.best_estimator_

# train error rate with best model
y_train_hat = best_.predict(X_train)
print("best estimator train error:", 1 - accuracy_score(y_train, y_train_hat))
print("best estimator train auc:", roc_auc_score(y_train, y_train_hat))
print("best estimator train precision:", precision_score(y_train, y_train_hat))

# test error rate with best model
y_test_hat = best_.predict(X_test)
print("best estimator test error:", 1 - accuracy_score(y_test, y_test_hat))
print("best estimator test auc:", roc_auc_score(y_test, y_test_hat))
print("best estimator test precision:", precision_score(y_test, y_test_hat))

print(confusion_table(y_train_hat, y_train))
print(confusion_table(y_test_hat, y_test))

In [ ]:
from sklearn.inspection import permutation_importance
perm_importance = permutation_importance(best_, X_train, y_train, n_repeats=30, random_state=seed, scoring='roc_auc')
best_features_idx = np.arange(X.shape[1])[perm_importance.importances_mean.argsort()][-2:]

In [ ]:
from matplotlib import font_manager
import matplotlib as mpl
# set theme

# use lato font
font_path = 'Reference/Lato/Lato-Regular.ttf'
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = prop.get_name()

# set colors
col_yellow = '#ffa400'
col_green = '#256143'
col_brown = '#9c5421'
col_off_black = '#434343'
mpl.rcParams['text.color'] = col_off_black # title and legend
mpl.rcParams['xtick.color'] = col_off_black # tick marks
mpl.rcParams['ytick.color'] = col_off_black # tick marks
mpl.rcParams['axes.labelcolor'] = col_off_black # axes labels

# set font sizes
mpl.rcParams['axes.titlesize'] = 24 # title
mpl.rcParams['axes.titleweight'] = 'bold' # title
mpl.rcParams['axes.labelsize'] = 18 # axes labels
mpl.rcParams['xtick.labelsize'] = 15 # tick marks
mpl.rcParams['ytick.labelsize'] = 15 # tick marks
mpl.rcParams['legend.title_fontsize'] = 18 # legend title
mpl.rcParams['legend.fontsize'] = 15 # legend text

# figure sizes (horizontal/vertical/square)
figsize_v = (6,10)
figsize_h = (10,6)

In [ ]:
feature_names = dataDF.drop(columns=["closed_since_2022", "longitude", "latitude"]).columns
feature_names = np.array([' '.join(f.split("_")) for f in feature_names])
sorted_idx = perm_importance.importances_mean.argsort()

# plot feature importance in box plots using the given theme
fig, ax = plt.subplots(figsize=figsize_v)
ax.boxplot(perm_importance.importances[sorted_idx[-10:]].T, vert=False, labels=feature_names[sorted_idx[-10:]], patch_artist=True, medianprops=dict(color=col_off_black), boxprops=dict(facecolor=col_yellow), widths=0.3)
title = plt.title("Top 10 Brewery Closure \n Feature Permutation Importances", loc="center")
title.set_position([0.2, 2])
plt.show()

In [ ]:
fig.savefig("Viz/Draft2/model_closure_svm.png", bbox_inches='tight')